# Veri setini dahil etme ve bağılı ve bağımsız değişlkenleri ayrıştırma

In [6]:
import pandas as pd 
df = pd.read_csv("Advertising.csv" ,usecols = [1 ,2 ,3 ,4 ] )
df = df.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [24]:
X = df.drop("sales" ,axis = 1)
y = df.sales

# Eğitim ve test seri oluşturma 

In [25]:
from sklearn.model_selection import train_test_split , cross_val_score , cross_val_predict

In [26]:
? train_test_split

Signature:  train_test_split(*arrays, **options)
Docstring:
Split arrays or matrices into random train and test subsets

Quick utility that wraps input validation and
``next(ShuffleSplit().split(X, y))`` and application to input data
into a single call for splitting (and optionally subsampling) data in a
oneliner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float, int or None, optional (default=None)
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

train_size : float, int, or None, (default=None)
    If float, should be between 0.0 

In [34]:
X_train , X_test , y_train , y_test = train_test_split(X , y ,test_size = 0.20 ,random_state = 10 )

In [35]:
X_train.shape

(160, 3)

In [36]:
X_test.shape

(40, 3)

In [37]:
y_train.shape

(160,)

In [38]:
y_test.shape

(40,)

# Statsmodel kurma işlemi 

In [46]:
import statsmodels.api as sm

In [93]:
model = sm.OLS(y_train ,X_train)
model

In [94]:
model = model.fit()
model

In [95]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.984
Model:                            OLS   Adj. R-squared (uncentered):              0.984
Method:                 Least Squares   F-statistic:                              3257.
Date:                Sat, 11 Apr 2020   Prob (F-statistic):                   4.23e-141
Time:                        12:58:28   Log-Likelihood:                         -324.87
No. Observations:                 160   AIC:                                      655.7
Df Residuals:                     157   BIC:                                      665.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0532      0.001     36.949      0.000       0.050       0.056
radio          0.2316      0.010     23.832      0.000       0.212       0.251
newspaper      0.0168      0.007      2.443      0.016       0.003       0.030
==============================================================================
Omnibus:                        4.968   Durbin-Watson:                   1.778
Prob(Omnibus):                  0.083   Jarque-Bera (JB):                4.176
Skew:                           0.302   Prob(JB):                        0.124
Kurtosis:                       2.488   Cond. No.                         12.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- Yukarıdaki R kare değerinin artmasının sebebi ,  bağımlı değişkenlerin bağımsız değişkenlerce açıklanabiliriliğinin artmasıdır 

# Sklearn ile model kurma 

In [96]:
from sklearn.linear_model import LinearRegression
# modeli import ediyoruz 

In [97]:
lm = LinearRegression()
lm
# modelden nesne oluşturuyoruz 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [102]:
lm = lm.fit(X_train , y_train)
lm
# model parametrelerini girip fit ediyoruz 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [99]:
lm.coef_
# b1 ,b2 ,b3 katsayıları 

array([ 0.0437726 ,  0.19343299, -0.00222879])

In [119]:
lm.intercept_ 
# b0 sabiti

3.2540971144188866

In [121]:
lm.score(X_train ,y_train)
#r kare değeri 


0.9209087553499528

### Tahminde bulunma 

Model denklemi : 
    Sales = 3.25 + TV  * 0.43  + radio * 0.19 + newspaper * -0.002

In [101]:
y_pred = lm.predict(X_test)
# elimizdeki test setini test ediyoruz 

In [112]:
y_pred = pd.DataFrame(y_pred)
# y_test ile karşılaştırabilmek için dataframeye dönüştürdüm 

### Hata karelerininin ortalamasının Karekökünü bulma 

In [115]:
from sklearn.metrics import mean_squared_error 
import numpy as np

In [117]:
rmse = np.sqrt(mean_squared_error(y_test , y_pred))
rmse
# burada y_test setimizin bağımlı değişkeni ile ,y_pred tahminlerinin hata karaler ortalamasının kare karekökünü bulduk


2.5885298446278107

### Model Turning  (Model doğrulama)

In [130]:
from sklearn.metrics import r2_score

In [136]:
lm.score(X_train , y_train)
# değişkenlerin veri setini açıklıyabilirliği 

0.9209087553499528

In [144]:
scores = cross_val_score(lm,
              X_train,
              y_train,
              cv = 10,
              scoring = "r2")
# Yukarada sadece  tek bir scrlama aldık şimdi ise
# 10 katlı çaprazlama yapıp bir scor elde ettik 

print(str(scores) ,"\n")
print("score mean : "  ,str(scores.mean()))

[0.89460976 0.93950986 0.80333388 0.94414693 0.8591572  0.92865633
 0.93605667 0.91725888 0.89502386 0.94632849] 

score mean :  0.9064081863045436


In [150]:
RMSE = np.sqrt(-cross_val_score(lm , # model'i giriyoruz
                X_train, 
                y_train,
                cv = 10 , # Kaç katlı caprazlama yapacaksak onu giriyoruz
                scoring = "neg_mean_squared_error" #istediğimiz hata türü
))
# burada değerlerimiz negatif geldiği için onune - işareti koyduk 
RMSE

array([1.60371977, 1.1944287 , 1.71353273, 1.2020478 , 1.35753673,
       1.40704009, 1.10421777, 1.39496092, 1.65223733, 1.07472994])

In [152]:
RMSE.mean()
# orlama hatamız 

1.3704451782615672